In [1]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "godfred_gameli.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "id": "1YRNpdG68VGM",
        "colab_type": "code",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 158
        },
        "outputId": "30f8e3e9-e29c-4cfc-d726-798467375494"
      },
      "source": [
        "!pip install tweet-preprocessor\n",
        "!pip install textblob\n",
        "!pip install tweet-preprocessor"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Collecting tweet-preprocessor\n",
            "  Downloading https://files.pythonhosted.org/packages/17/9d/71bd016a9edcef8860c607e531f30bd09b13103c7951ae73dd2bf174163c/tweet_preprocessor-0.6.0-py3-none-any.whl\n",
            "Installing collected packages: tweet-preprocessor\n",
            "Successfully installed tweet-preprocessor-0.6.0\n",
            "Requirement already satisfied: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)\n",
            "Requirement already satisfied: nltk>=3.1 in /usr/local/lib/python3.6/dist-packages (from textblob) (3.2.5)\n",
            "Requirement already satisfied: six in /usr/local/lib/python3.6/dist-packages (from nltk>=3.1->textblob) (1.12.0)\n",
            "Requirement already satisfied: tweet-preprocessor in /usr/local/lib/python3.6/dist-packages (0.6.0)\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "B0Uh5g2Y8Uhf",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "import sys\n",
        "import os\n",
        "import json\n",
        "import pandas as pd\n",
        "import matplotlib.pyplot as plt"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "syT4ggI88a7i",
        "colab_type": "code",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 87
        },
        "outputId": "2d6dfacc-9a48-415c-9e4b-06d956f58bd7"
      },
      "source": [
        "#Import the necessary methods from tweepy library  \n",
        "\n",
        "#install tweepy if you don't have it\n",
        "#!pip install tweepy\n",
        "import tweepy\n",
        "from tweepy.streaming import StreamListener\n",
        "from tweepy import OAuthHandler\n",
        "from tweepy import Stream\n",
        "\n",
        "#sentiment analysis package\n",
        "#!pip install textblob\n",
        "from textblob import TextBlob\n",
        "\n",
        "#general text pre-processor\n",
        "#!pip install nltk\n",
        "import nltk\n",
        "from nltk.corpus import stopwords\n",
        "nltk.download('stopwords')\n",
        "nltk.download('punkt')\n",
        "from nltk.tokenize import word_tokenize\n",
        "\n",
        "#tweet pre-processor \n",
        "#!pip install tweet-preprocessor\n",
        "import preprocessor as ppr\n",
        "\n",
        "import twitter_credentials\n",
        "import re\n",
        "import string"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "[nltk_data] Downloading package stopwords to /root/nltk_data...\n",
            "[nltk_data]   Unzipping corpora/stopwords.zip.\n",
            "[nltk_data] Downloading package punkt to /root/nltk_data...\n",
            "[nltk_data]   Unzipping tokenizers/punkt.zip.\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "5XxyUHTv7l1_",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "class tweetsearch():\n",
        "    '''\n",
        "    This is a basic class to search and download twitter data.\n",
        "    You can build up on it to extend the functionalities for more \n",
        "    sophisticated analysis\n",
        "    '''\n",
        "    def __init__(self,cols=None,auth=None):\n",
        "        #\n",
        "        if not cols is None:\n",
        "            self.cols = cols\n",
        "        else:\n",
        "            self.cols = ['id', 'created_at', 'source', 'original_text','clean_text', \n",
        "                    'sentiment','polarity','subjectivity', 'lang',\n",
        "                    'favorite_count', 'retweet_count', 'original_author',   \n",
        "                    'possibly_sensitive', 'hashtags',\n",
        "                    'user_mentions', 'place', 'place_coord_boundaries']\n",
        "            \n",
        "        if auth is None:\n",
        "            #Variables that contains the user credentials to access Twitter API \n",
        "            #consumer_key = os.environ.get('TWITTER_API_KEY')\n",
        "            #consumer_secret = os.environ.get('TWITTER_API_SECRET')\n",
        "            #access_token = os.environ.get('TWITTER_ACCESS_TOKEN')\n",
        "            #access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')\n",
        "\n",
        "\n",
        "            #This handles Twitter authetification and the connection to Twitter Streaming API\n",
        "            \n",
        "            #auth = OAuthHandler(consumer_key, consumer_secret)\n",
        "            #auth.set_access_token(access_token, access_token_secret)\n",
        "\n",
        "            auth=OAuthHandler(twitter_credentials.CONSUMER_KEY,twitter_credentials.CONSUMER_SECRET)\n",
        "            auth.set_access_token(twitter_credentials.ACCES_TOKEN,twitter_credentials.ACCES_TOKEN_SECRET)\n",
        "            \n",
        "\n",
        "        #            \n",
        "        self.auth = auth \n",
        "        self.api= tweepy.API(self.auth)            \n",
        "      \n",
        "\n",
        "    def clean_tweets(self,twitter_text):\n",
        "\n",
        "        #use pre processor\n",
        "        tweet = ppr.clean(twitter_text)\n",
        "\n",
        "         #HappyEmoticons\n",
        "        emoticons_happy = set([\n",
        "            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',\n",
        "            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',\n",
        "            '=-3', '=3', ':-))', \":'-)\", \":')\", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',\n",
        "            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',\n",
        "            '<3'\n",
        "            ])\n",
        "\n",
        "        # Sad Emoticons\n",
        "        emoticons_sad = set([\n",
        "            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',\n",
        "            ':-[', ':-<', '=\\\\', '=/', '>:(', ':(', '>.<', \":'-(\", \":'(\", ':\\\\', ':-c',\n",
        "            ':c', ':{', '>:\\\\', ';('\n",
        "            ])\n",
        "\n",
        "        #Emoji patterns\n",
        "        emoji_pattern = re.compile(\"[\"\n",
        "                 u\"\\U0001F600-\\U0001F64F\"  # emoticons\n",
        "                 u\"\\U0001F300-\\U0001F5FF\"  # symbols & pictographs\n",
        "                 u\"\\U0001F680-\\U0001F6FF\"  # transport & map symbols\n",
        "                 u\"\\U0001F1E0-\\U0001F1FF\"  # flags (iOS)\n",
        "                 u\"\\U00002702-\\U000027B0\"\n",
        "                 u\"\\U000024C2-\\U0001F251\"\n",
        "                 \"]+\", flags=re.UNICODE)\n",
        "\n",
        "        #combine sad and happy emoticons\n",
        "        emoticons = emoticons_happy.union(emoticons_sad)\n",
        "\n",
        "        stop_words = set(stopwords.words('english'))\n",
        "        stop_words.update(['#coronavirus', '#coronavirusoutbreak', '#coronavirusPandemic', '#covid19', '#covid_19', '#epitwitter', '#ihavecorona', 'amp', 'coronavirus', 'covid19','clean_text'])\n",
        "\n",
        "        word_tokens = word_tokenize(tweet)\n",
        "        #after tweepy preprocessing the colon symbol left remain after      \n",
        "        #removing mentions\n",
        "        tweet = re.sub(r':', '', tweet)\n",
        "        tweet = re.sub(r'‚Ä¶', '', tweet)\n",
        "\n",
        "        #replace consecutive non-ASCII characters with a space\n",
        "        tweet = re.sub(r'[^\\x00-\\x7F]+',' ', tweet)\n",
        "\n",
        "        #remove emojis from tweet\n",
        "        tweet = emoji_pattern.sub(r'', tweet)\n",
        "\n",
        "        #filter using NLTK library append it to a string\n",
        "        filtered_tweet = [w for w in word_tokens if not w in stop_words]\n",
        "\n",
        "        #looping through conditions\n",
        "        filtered_tweet = []    \n",
        "        for w in word_tokens:\n",
        "        #check tokens against stop words , emoticons and punctuations\n",
        "            if w not in stop_words and w not in emoticons and w not in string.punctuation:\n",
        "                filtered_tweet.append(w)\n",
        "\n",
        "        return ' '.join(filtered_tweet)            \n",
        "\n",
        "\n",
        "    def get_tweets(self,keyword, csvfile=None):\n",
        "        \n",
        "        #df = pd.DataFrame(columns=self.cols)\n",
        "        \n",
        "        #if  csvfile is None:\n",
        "        #If the file exists, then read the existing data from the CSV file.\n",
        "        if os.path.exists(csvfile):\n",
        "            df = pd.read_csv(csvfile, header=None)\n",
        "        else:\n",
        "            df=pd.DataFrame(columns=self.cols)\n",
        "            \n",
        "\n",
        "        #page attribute in tweepy.cursor and iteration\n",
        "        for page in tweepy.Cursor(self.api.search, q=keyword,count=200, include_rts=False).pages():\n",
        "\n",
        "            for status in page:\n",
        "                \n",
        "                new_entry = []\n",
        "                status = status._json\n",
        "                \n",
        "                #filter by language\n",
        "                if status['lang'] != 'en':\n",
        "                    continue\n",
        "\n",
        "                \n",
        "                #if this tweet is a retweet update retweet count\n",
        "                if status['created_at'] in df['created_at'].values:\n",
        "                    i = df.loc[df['created_at'] == status['created_at']].index[0]\n",
        "                    #\n",
        "                    cond1 = status['favorite_count'] != df.at[i, 'favorite_count']\n",
        "                    cond2 = status['retweet_count'] != df.at[i, 'retweet_count']\n",
        "                    if cond1 or cond2:\n",
        "                        df.at[i, 'favorite_count'] = status['favorite_count']\n",
        "                        df.at[i, 'retweet_count'] = status['retweet_count']\n",
        "                    continue\n",
        "                #print(status['text'])\n",
        "\n",
        "                #calculate sentiment\n",
        "                filtered_tweet =self.clean_tweets(status['text'])\n",
        "                blob = TextBlob(filtered_tweet)\n",
        "                Sentiment = blob.sentiment     \n",
        "                polarity = Sentiment.polarity\n",
        "                subjectivity = Sentiment.subjectivity\n",
        "\n",
        "                new_entry += [status['id'], status['created_at'],\n",
        "                              status['source'], status['text'],filtered_tweet, \n",
        "                              Sentiment,polarity,subjectivity, status['lang'],\n",
        "                              status['favorite_count'], status['retweet_count']]\n",
        "\n",
        "                new_entry.append(status['user']['screen_name'])\n",
        "\n",
        "                try:\n",
        "                    is_sensitive = status['possibly_sensitive']\n",
        "                except KeyError:\n",
        "                    is_sensitive = None\n",
        "\n",
        "                new_entry.append(is_sensitive)\n",
        "\n",
        "                hashtags = \", \".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])\n",
        "                new_entry.append(hashtags) #append the hashtags\n",
        "\n",
        "                #\n",
        "                mentions = \", \".join([mention['screen_name'] for mention in status['entities']['user_mentions']])\n",
        "                new_entry.append(mentions) #append the user mentions\n",
        "\n",
        "                try:\n",
        "                    xyz = status['place']['bounding_box']['coordinates']\n",
        "                    coordinates = [coord for loc in xyz for coord in loc]\n",
        "                except TypeError:\n",
        "                    coordinates = None\n",
        "                #\n",
        "                new_entry.append(coordinates)\n",
        "\n",
        "                try:\n",
        "                    location = status['user']['location']\n",
        "                except TypeError:\n",
        "                    location = ''\n",
        "                #\n",
        "                new_entry.append(location)\n",
        "                \n",
        "                #now append a row to the dataframe\n",
        "                single_tweet_df = pd.DataFrame([new_entry],columns=self.cols)\n",
        "                df_final = df.append(single_tweet_df, ignore_index=True)\n",
        "            \n",
        "            #save it to file\n",
        "            csvFile = open(csvfile, 'a' ,encoding='utf-8')\n",
        "            df_final.to_csv(csvFile,mode='a',columns=self.cols, encoding=\"utf-8\")\n",
        "          \n",
        "         \n",
        "        return df_final\n",
        "      \n",
        "   \n"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "xBTVKcf575O0",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        "covid_keywords = '#COVID19Ethiopia OR #COVID19Africa'  #hashtag based search\n",
        "#tweets_file = 'opia_covid19_23june2020.json'\n",
        "tweets_file='ethiopia_covid19_23june2020.csv'\n",
        "\n",
        "\n",
        "\n",
        "#get data on keywords\n",
        "\n",
        "ts = tweetsearch()\n",
        "df = ts.get_tweets(covid_keywords, csvfile=tweets_file) \n",
        "   #you saved the "
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "l0EyRm9Y8H7M",
        "colab_type": "code",
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 164
        },
        "outputId": "b0b2b74d-ab13-4bbd-9cb3-2fb9c5ce36f3"
      },
      "source": [
        "df.head(5)"
      ],
      "execution_count": null,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/html": [
              "<div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th></th>\n",
              "      <th>id</th>\n",
              "      <th>created_at</th>\n",
              "      <th>source</th>\n",
              "      <th>original_text</th>\n",
              "      <th>clean_text</th>\n",
              "      <th>sentiment</th>\n",
              "      <th>polarity</th>\n",
              "      <th>subjectivity</th>\n",
              "      <th>lang</th>\n",
              "      <th>favorite_count</th>\n",
              "      <th>retweet_count</th>\n",
              "      <th>original_author</th>\n",
              "      <th>possibly_sensitive</th>\n",
              "      <th>hashtags</th>\n",
              "      <th>user_mentions</th>\n",
              "      <th>place</th>\n",
              "      <th>place_coord_boundaries</th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>0</th>\n",
              "      <td>1275067907710443524</td>\n",
              "      <td>Mon Jun 22 14:07:33 +0000 2020</td>\n",
              "      <td>&lt;a href=\"http://twitter.com/download/android\" ...</td>\n",
              "      <td>RT @Amref_Worldwide: Why wear a mask? Is it ne...</td>\n",
              "      <td>Why wear mask Is necessary Is required everyon...</td>\n",
              "      <td>(0.13333333333333333, 0.5666666666666667)</td>\n",
              "      <td>0.133333</td>\n",
              "      <td>0.566667</td>\n",
              "      <td>en</td>\n",
              "      <td>0</td>\n",
              "      <td>484</td>\n",
              "      <td>LGwara</td>\n",
              "      <td>None</td>\n",
              "      <td></td>\n",
              "      <td>Amref_Worldwide, daktari1</td>\n",
              "      <td>None</td>\n",
              "      <td>Nairobi</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "</div>"
            ],
            "text/plain": [
              "                    id  ... place_coord_boundaries\n",
              "0  1275067907710443524  ...                Nairobi\n",
              "\n",
              "[1 rows x 17 columns]"
            ]
          },
          "metadata": {
            "tags": []
          },
          "execution_count": 8
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "qgJLVXmR8pGW",
        "colab_type": "code",
        "colab": {}
      },
      "source": [
        ""
      ],
      "execution_count": null,
      "outputs": []
    }
  ]
} 

NameError: ignored